# Loading dependencies and data files 

In [2]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import svm
from sklearn.model_selection import cross_val_score


fake, non_fake  = pd.read_excel('cleaned_files/Fakeset_cleaned_165.xlsx'), pd.read_excel('cleaned_files/Non-fakeset_cleaned_318.xlsx')

dataset = pd.concat([fake, non_fake])

pravda_unian_set = pd.read_excel('cleaned_files/Non-fakeset.xlsx')

testing_set = pravda_unian_set[(pravda_unian_set['url'].isin(dataset['url']) == False)]

dataset.reset_index(np.linspace(1,len(dataset)), inplace=True)

# Instantiating word vectorizer + stopwords

In [3]:
stemmer = SnowballStemmer(language='russian')

analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc) if w not in rus_stopwords)

rus_stopwords = ["c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]


# Model training

In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset['ru_text'], 
                                                    dataset['target'], 
                                                    random_state=0)

vect = CountVectorizer(min_df=5, ngram_range=(1,3), analyzer=stemmed_words).fit(X_train)

X_train_vectorized = vect.transform(X_train)

model = svm.SVC(kernel='linear').fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

auc = roc_auc_score(y_test, predictions)

accuracy = model.score(vect.transform(X_test), y_test)

# Testing

In [5]:
testing = vect.transform(testing_set['ru_text'])

res = model.predict(testing)

'''By using sole word vectorizers, one can achieve almost 100% accuracy for test set. 
However, test set from the same news sources (Ukrainska Pravda and UNIAN), the score drops to 72-77% accuracy'''
print({'accuracy': accuracy, 'auc': auc, 'test_set_accuracy': (1-np.count_nonzero(res)/len(res))})

{'accuracy': 0.98347107438016534, 'auc': 0.98022875816993471, 'test_set_accuracy': 0.776173285198556}


# Cross-validation

In [7]:
X, y = CountVectorizer(min_df=5, ngram_range=(1,3), analyzer=stemmed_words).fit_transform(dataset['ru_text']), dataset['target']

clf = svm.SVC(kernel='linear', random_state=0)

scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')

print('Cross-validation (accuracy)', cross_val_score(clf, X, y, cv=5))

print('Cross-validation (AUC)', cross_val_score(clf, X, y, cv=5, scoring = 'roc_auc'))

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Cross-validation (accuracy) [ 0.95876289  0.92783505  0.96907216  0.94791667  0.91666667]
Cross-validation (AUC) [ 0.94602273  0.96543561  0.99573864  0.98412698  0.98941799]
Accuracy: 0.94 (+/- 0.04)


# Top 20 words by frequencies 

## Ukrainian news

In [9]:
X_train_vectorized_positive = X_train_vectorized[np.array(y_train.astype('bool')), :]
X_train_vectorized_negative = X_train_vectorized[~np.array(y_train.astype('bool')), :]


feature_names = np.array(vect.get_feature_names())

sorted_tfidf_index = X_train_vectorized_negative.max(0).toarray()[0].argsort()

values = X_train_vectorized_negative.max(0).toarray()[0]

small_idf = pd.Series(values[sorted_tfidf_index[:20]], index=feature_names[sorted_tfidf_index[:20]]).sort_index()

large_idf = pd.Series(values[sorted_tfidf_index[:-21:-1]], index=feature_names[sorted_tfidf_index[:-21:-1]]).sort_index()



## Russian news

In [10]:
feature_names = np.array(vect.get_feature_names())

sorted_tfidf_index = X_train_vectorized_positive.max(0).toarray()[0].argsort()

values = X_train_vectorized_negative.max(0).toarray()[0]

small_idf = pd.Series(values[sorted_tfidf_index[:20]], index=feature_names[sorted_tfidf_index[:20]]).sort_index()

large_idf = pd.Series(values[sorted_tfidf_index[:-21:-1]], index=feature_names[sorted_tfidf_index[:-21:-1]]).sort_index()
